In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torchvision
import torch
from matplotlib.patches import Rectangle
import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

import sys
print("Python 环境路径:", sys.prefix)
print("Python 版本:", sys.version)
print("Python 可执行文件路径:", sys.executable)

In [ ]:
# 加载模型，读取签名
new_model=tf.saved_model.load("/home/jhr/Program/TensorFlow/00-model/model/last_saved_model-3")
model_signature=new_model.signatures["serving_default"]
print(model_signature)
#查看模型结构
model_signature.inputs,model_signature.outputs

In [ ]:
img=tf.io.read_file("/home/jhr/Program/TensorFlow/03-program/cap1_15.jpg")
img=tf.image.decode_jpeg(img,channels=3)
img=tf.image.resize(img,[640,640])
img=img/255.0
plt.imshow(img)

In [ ]:
dataset=tf.constant(img)
print("shape1 ",dataset.shape)
dataset=np.expand_dims(dataset,0)
print("shape2 ",dataset.shape)

In [ ]:
resout=model_signature(x=dataset)#模型预测
resout=resout["output_0"]
resout=resout.numpy()
resout[0][...,:4]*=[640,640,640,640]

In [ ]:
agnostic=False
conf_thres=0.25
iou_thres=0.45
max_wh = 7680
max_nms = 30000
max_det=300
nm=0
bs=resout.shape[0] #batch_size
nc=resout.shape[2]-5    #number of classes
xc=resout[...,4]>conf_thres     #condidates
mi=10
output=[tf.zeros((0,6))]*bs

In [ ]:
#tensorflow模式
def xywh2xyxy(x):
    y=np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2] / 2.0 # top left x
    y[..., 1] = x[..., 1] - x[..., 3] / 2.0  # top left y
    y[..., 2] = x[..., 0] + x[..., 2] / 2.0  # bottom right x
    y[..., 3] = x[..., 1] + x[..., 3] / 2.0  # bottom right y
    return y

for xi,x in enumerate(resout): #image index,image inference
    x=x[xc[xi]]#根据阈值剔除不符合的数据【116，10】
    x[:,5:]*=x[:,4:5]
    box = xywh2xyxy(x[:, :4])#center_x, center_y, width, height) to (x1, y1, x2, y2)
    mask=x[:,mi:]# zero columns if no masks

    conf = tf.reduce_max(x[:,5:mi], axis=1, keepdims=True)#计算每一列最大值
    j = np.expand_dims(tf.math.argmax(x[:,5:mi], axis=1),-1)#保存每列最大值的索引

    # #拼接张量
    concatenated = tf.concat([box, conf, j, mask], axis=1)
    conf_mask=tf.reshape(conf,[-1])>conf_thres#展平矩阵，并与conf_thres比较
    x=tf.boolean_mask(concatenated,conf_mask,axis=0)#center_x, center_y, width, height，percent，class

    #将x按照x[:,4]进行排列
    sorted_indices=tf.argsort(x[:,4],direction="DESCENDING")
    x=tf.gather(x,sorted_indices[:max_nms])

    c = x[:, 5:6] * max_wh
    boxes, scores = x[:, :4] + c, x[:, 4]#boxes是
    #极大值抑制算法
    i=tf.image.non_max_suppression(boxes,scores,iou_threshold=iou_thres,max_output_size=10)
    i=i[:max_det]
    output[xi]=tf.gather(x,i)
output

In [ ]:
#pytorch模式
def xywh2xyxy(x):
    y=np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2] / 2.0 # top left x
    y[..., 1] = x[..., 1] - x[..., 3] / 2.0  # top left y
    y[..., 2] = x[..., 0] + x[..., 2] / 2.0  # bottom right x
    y[..., 3] = x[..., 1] + x[..., 3] / 2.0  # bottom right y
    return torch.tensor(y)

#torch模式
for xi,x in enumerate(resout):
    x=torch.tensor(x)#抓换为tensor对象
    x=x[xc[xi]]
    x[:,5:]*=x[:,4:5]
    box=xywh2xyxy(x[:,:4])#计算四个边框
    mask=x[:,mi:]
    conf, j = x[:, 5:mi].max(1, keepdim=True)
    x = torch.cat((box, conf, j.float(), mask), 1)[conf.view(-1) > conf_thres]
    x = x[x[:, 4].argsort(descending=True)[:max_nms]]
    c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
    boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
    i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS 非最大值抑制
    i = i[:max_det]  # limit detections
    output[xi] = x[i]

# output

In [ ]:
rect=output[0][3,:4]
plt.imshow(img)
rect=Rectangle((rect[0],rect[1]),
               (rect[2]-rect[0]),
               (rect[3]-rect[1]),
               fill=False,
               color='red')
ax=plt.gca()
ax.axes.add_patch(rect)
plt.show()